In [10]:
import pandas as pd
import json
import numpy as np

In [11]:
file = 'attention_matrices/metadata.json'

with open(file, 'r') as f:
    data = json.load(f)
df = pd.DataFrame(data)
df.columns


Index(['attention_matrix_path', 'max_nodes', 'num_nodes', 'num_edges',
       'connection_probability', 'layer', 'head', 'source', 'target', 'tokens',
       'prompt'],
      dtype='object')

In [13]:
# df['num_nodes'].value_counts()
df['num_edges'].value_counts()

num_edges
21    33
20    31
19    31
17    27
22    25
18    23
23    22
16    22
25    17
24    12
15    10
27     9
26     7
14     7
28     6
29     5
30     4
13     3
12     2
10     2
9      2
Name: count, dtype: int64